In [1]:
#Importar bibliotecas
import pandas as pd
import numpy as np
import psycopg2 as pg2

In [ ]:
#select fullvisitorid, 
#SUM(revenue) AS Receita,
#COUNT(fullvisitorid) AS qt_de_transações,
#AVG(revenue) AS media_por_transação
#FROM hits
#GROUP BY (fullvisitorid)